## Computer Vision

1. Face Recognition:
Use CV to recognize faces for user's logins validation.

2. Image Similarity Search:
Use CV to search images with other image as query and find similar images.

## Face Recognition

In [8]:
from facenet_pytorch import InceptionResnetV1, MTCNN
import torch
import PIL

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device 

'cpu'

In [9]:
import PIL.Image


img = PIL.Image.open("../assets/me.png")

In [7]:
mtcnn = MTCNN(image_size=160, margin=5, post_process=False, device=device)
mt